# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# 03 – Drop-In Knowledge Mode: Context Builder (RAG) Demo

This notebook demonstrates **how to wire the new `ContextBuilder` into the Drop-In Knowledge Mode runtime**, using only the minimal set of components that already exist in the Intergrax framework. :contentReference[oaicite:0]{index=0}

The goals of this notebook are:

1. **Load a demo chat session**  
   - Create or load a `ChatSession` using the existing `SessionStore`.
   - Add a few example `ChatMessage` objects to simulate a short conversation.

2. **Work with an existing attachment**  
   - Use an attachment that has already been ingested (from the previous notebook `02_attachments_ingestion_demo.ipynb`), or prepare a single demo attachment (e.g. `PROJECT_STRUCTURE.md`) and ingest it using the existing ingestion pipeline.
   - Make sure the attachment’s chunks are stored in the vector store with proper metadata (including `session_id` and `attachment_id`).

3. **Initialize the `ContextBuilder`**  
   - Construct a `ContextBuilder` instance using:
     - the same `RuntimeConfig` as the Drop-In Knowledge Mode runtime,
     - the shared `IntergraxVectorstoreManager`.
   - Briefly explain the configuration knobs that affect RAG (e.g. `max_docs_per_query`, `max_history_messages`, optional score threshold).

4. **Build context for a single user question**  
   - Create a `RuntimeRequest` that represents the current user query.
   - Call `ContextBuilder.build_context(session, request)` to obtain:
     - reduced chat history,
     - retrieved document chunks (RAG context),
     - system prompt,
     - RAG debug information.

5. **Inspect the result of context building (no LLM call yet)**  
   - Print and inspect:
     - the reduced chat history selected by the `ContextBuilder`,
     - the list of retrieved chunks (text, metadata, score),
     - the internal RAG debug info (filters used, scores, etc.),
     - the final system prompt that would be sent to the LLM.
   - Optionally, show how these pieces can be turned into a list of messages for the LLM, but **without** modifying the main runtime engine yet.

6. **Prepare for engine integration (next step)**  
   - Clearly separate what is done in this notebook (manual usage of `ContextBuilder`) from what will be done in the next step:
     - integrating `ContextBuilder` into `DropInKnowledgeRuntime.ask()`,
     - wiring `rag_debug_info` into the runtime’s `debug_trace`.

By the end of this notebook, we will have a **practical, end-to-end demonstration** of how the `ContextBuilder` uses:
- session history,
- RAG retrieval from the vector store,
- and runtime configuration

to produce a ready-to-use context object that the Drop-In Knowledge Mode engine can pass to any LLM adapter.


## 1) Runtime Initialization

This cell initializes the minimum set of components required to test the `ContextBuilder`.

We are **not testing ingestion here** — attachments are assumed to be already ingested in the previous notebook (`02_attachments_ingestion_demo.ipynb`).

Components initialized in this step:

- In-memory session store
- LLM adapter (Ollama-based)
- Embedding manager (same model as ingestion pipeline)
- Vector store manager (Chroma, same collection as before)
- Runtime config (RAG will be enabled in the next cell)
- Drop-In Knowledge runtime (used only as a dependency context)

In [ ]:
import sys, os

from intergrax.llm_adapters.llm_provider import LLMAdapterRegistry, LLMProvider
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.globals.settings import GLOBAL_SETTINGS
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage


from datetime import datetime
from pathlib import Path


from intergrax.llm.messages import ChatMessage

from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VectorstoreManager, VSConfig

from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


# 1) Session store – temporary storage for chat messages and metadata
session_manager = SessionManager(
    storage=InMemorySessionStorage()
)

# 2) LLM adapter (Ollama backend)
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# 3) Embedding manager (must match ingestion model)
embedding_manager = EmbeddingManager(
    provider="ollama",
    model_name=GLOBAL_SETTINGS.default_ollama_embed_model,
    assume_ollama_dim=1536,
)

# 4) Vector store connection (same collection as ingestion)
vectorstore_cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs_drop_in_demo",
)

vectorstore_manager = VectorstoreManager(
    config=vectorstore_cfg,
    verbose=True,
)

# 5) Runtime config – here we will enable RAG shortly
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    embedding_manager=embedding_manager,
    vectorstore_manager=vectorstore_manager,
    enable_rag=True,               # <-- ACTIVATED HERE (previous notebook had it off)
    enable_websearch=False,
    tools_mode="off",
    enable_user_profile_memory=True,
)

# 6) Drop-In Knowledge Mode runtime
runtime = DropInKnowledgeRuntime(
    config=config,
    session_manager=session_manager,
)

print("Runtime initialized at", datetime.utcnow().isoformat())



[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_docs_drop_in_demo
[intergraxVectorstoreManager] Existing count: 100
Runtime initialized at 2025-11-25T17:30:24.581366


C:\Users\XPS\AppData\Local\Temp\ipykernel_17444\2805045218.py:67: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Runtime initialized at", datetime.utcnow().isoformat())


## 2) Prepare a demo chat session and minimal history

In this step we:

1. Create a new in-memory chat session for a demo user.
2. Append a few simple messages to simulate an ongoing conversation.
3. Verify that the session object contains:
   - a `session_id` (or equivalent),
   - `user_id`, `tenant_id`, `workspace_id`,
   - a small list of `ChatMessage` objects.

This session will later be passed to the `ContextBuilder` together with a `RuntimeRequest`.


In [ ]:
# 2) Prepare a demo chat session and a minimal message history

# NOTE:
# The exact API of SessionStore may differ slightly in your codebase.
# This example assumes a simple create_session(...) + get_session(...) + append_message(...) flow.
# If your implementation uses different method names or signatures,
# please adjust the calls below accordingly.

# 2.1 Create a new session for a demo user
demo_user_id = "demo-user-context-builder"
demo_tenant_id = "demo-tenant"
demo_workspace_id = "demo-workspace"

session = await session_manager.create_session(
    user_id=demo_user_id,
    tenant_id=demo_tenant_id,
    workspace_id=demo_workspace_id,
)

print("Created session:")
print("  session_id :", getattr(session, "session_id", getattr(session, "id", "<no-id-attr>")))
print("  user_id    :", getattr(session, "user_id", None))
print("  tenant_id  :", getattr(session, "tenant_id", None))
print("  workspace  :", getattr(session, "workspace_id", None))

# 2.2 Append a couple of messages to simulate conversation history
await session_manager.append_message(
    session_id=session.id,
    message=ChatMessage(role="user", content="Hi, I want to explore the Intergrax framework."),
)

await session_manager.append_message(
    session_id=session.id,
    message=ChatMessage(role="assistant", content="Sure, I can help you with that."),
)

session = await session_manager.get_session(session.id)
messages = await session_manager._storage.get_history(session_id=session.id)

print("\nSession messages:")
for msg in messages:
    print(f"- [{msg.role}] {msg.content}")


Created session:
  session_id : b087d6de-3cda-4d0e-a26d-f229fd657118
  user_id    : demo-user-context-builder
  tenant_id  : demo-tenant
  workspace  : demo-workspace

Session messages:
- [user] Hi, I want to explore the Intergrax framework.
- [assistant] Sure, I can help you with that.


## 3) Initialize `ContextBuilder`, ingest the attachment for this session, and build context

In this step we:

1. Recreate the same `AttachmentRef` that was used in the ingestion demo  
   (here: `PROJECT_STRUCTURE.md` at the repository root).
2. Send a one-shot `RuntimeRequest` with this attachment to `DropInKnowledgeRuntime.ask(...)`  
   so that:
   - the engine stores a message with the attachment in the current session,
   - the ingestion pipeline runs and upserts document chunks into the vector store
     with this session's metadata.
3. Initialize the `ContextBuilder` using the same `RuntimeConfig` and `VectorstoreManager`.
4. Build context for a real user question using `ContextBuilder.build_context(session, request)`.

The result will be stored in `built_context` and inspected in the next section.


In [ ]:
from intergrax.llm.messages import AttachmentRef
from intergrax.runtime.drop_in_knowledge_mode.context.context_builder import BuiltContext, ContextBuilder
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest

# 3.1 Recreate the attachment used in the ingestion demo (PROJECT_STRUCTURE.md)
project_root_file = Path("../../PROJECT_STRUCTURE.md").resolve()

if not project_root_file.exists():
    raise FileNotFoundError(f"Expected demo file does not exist: {project_root_file}")

attachment_uri = project_root_file.as_uri()

attachment = AttachmentRef(
    id="project-structure-md",
    type="markdown",
    uri=attachment_uri,
    metadata={
        "description": "Intergrax project structure document",
        "scope": "intergrax-docs-demo",
    },
)

print("AttachmentRef recreated:")
print("  id       :", attachment.id)
print("  type     :", attachment.type)
print("  uri      :", attachment.uri)


# 3.2 Send a one-shot request with this attachment to the runtime
# NOTE:
# This call should:
# - append a new user message (with the attachment) to the session,
# - run the ingestion pipeline for THIS session,
# - push vectors to the configured vector store with proper metadata.
ingestion_request = RuntimeRequest(
    session_id=session.id,
    user_id=session.user_id,
    tenant_id=session.tenant_id,
    workspace_id=session.workspace_id,
    message="Please index this attachment for the current session.",
    attachments=[attachment],
)

print("\nSending ingestion request to DropInKnowledgeRuntime...")
ingestion_answer = await runtime.run(ingestion_request)
print("Ingestion request completed.")
print("Answer summary:", ingestion_answer.stats)


# 3.3 Reload the session to ensure the new message (with attachment) is stored
session = await session_manager.get_session(session.id)
messages = await session_manager._storage.get_history(session_id=session.id)
print("\nSession now has", len(messages), "message(s).")


# 3.4 Initialize ContextBuilder (RAG is enabled in RuntimeConfig)
context_builder = ContextBuilder(
    vectorstore_manager=vectorstore_manager,
    config=config,
)

print("\nContextBuilder initialized.")


# 3.5 Create a RuntimeRequest representing the actual user question
request = RuntimeRequest(
    session_id=session.id,
    user_id=session.user_id,
    tenant_id=session.tenant_id,
    workspace_id=session.workspace_id,
    message="Can you summarize the key components of the Intergrax architecture?",
)

print("RuntimeRequest prepared.")
print("User query:", request.message)


# 3.6 Build context using the current session + user question
built_context : BuiltContext = await context_builder.build_context(
    session=session,
    request=request,
)

print("\nContext built.")
print("RAG used        :", built_context.rag_debug_info.get("used"))
print("Chunks retrieved:", len(built_context.retrieved_chunks))


AttachmentRef recreated:
  id       : project-structure-md
  type     : markdown
  uri      : file:///D:/Projekty/intergrax/PROJECT_STRUCTURE.md

Sending ingestion request to DropInKnowledgeRuntime...
[intergraxVectorstoreManager] Upserting 100 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 100
Ingestion request completed.
Answer summary: ok

Session now has 4 message(s).

ContextBuilder initialized.
RuntimeRequest prepared.
User query: Can you summarize the key components of the Intergrax architecture?

Context built.
RAG used        : True
Chunks retrieved: 8


## 4) Inspect the built context (history, retrieved chunks, debug info)

Now that we have a `BuiltContext` instance, we want to inspect what the
`ContextBuilder` actually produced:

1. **System prompt** – the instruction that will be sent as the `system` message.
2. **Reduced chat history** – messages that will be forwarded to the LLM.
3. **Retrieved chunks (RAG context)** – text fragments loaded from the vector store.
4. **RAG debug info** – internal diagnostics (filters used, scores, etc.).

This step is purely diagnostic and does **not** call the LLM yet.
It helps verify that the RAG layer is working as expected before we integrate
it into `DropInKnowledgeRuntime.ask()`.


In [ ]:
from pprint import pprint

messages = await session_manager._storage.get_history(session_id=session.id)

# 4.1 Inspect reduced chat history
print("=== REDUCED CHAT HISTORY ===")
if not messages:
    print("(no history messages)")
else:
    for i, msg in enumerate(messages, start=1):
        print(f"[{i}] role={msg.role}")
        print(msg.content)
        print("-" * 60)
print("\n")


# 4.2 Inspect retrieved chunks (RAG context)
print("=== RETRIEVED CHUNKS (RAG CONTEXT) ===")
chunks = built_context.retrieved_chunks

print(f"Total retrieved chunks: {len(chunks)}\n")

if not chunks:
    print("(no chunks retrieved - check if your vector store contains data and the metadata filters are correct)")
else:
    max_preview = 3  # limit output for readability
    for idx, ch in enumerate(chunks[:max_preview], start=1):
        print(f"--- CHUNK #{idx} ---")
        print("ID    :", ch.id)
        print("Score :", ch.score)
        print("Meta  :", ch.metadata)
        print("\nTEXT PREVIEW:\n")
        preview = ch.text[:500] if ch.text else ""
        print(preview)
        print("\n" + "=" * 80 + "\n")


# 4.3 Inspect RAG debug info
print("=== RAG DEBUG INFO ===")
pprint(built_context.rag_debug_info)


=== SYSTEM PROMPT ===
You are an AI assistant running in the Intergrax Drop-In Knowledge Mode.

The runtime automatically:
- receives and processes any user attachments (files),
- splits them into chunks, indexes them in a vector store,
- retrieves only the most relevant chunks for the current question.

You see the final retrieved context as plain text snippets. You do NOT need to talk about indexing, vector stores, embeddings, or technical ingestion steps.

Important behavior rules:
1) Never say that you cannot see or access attachments. The runtime has already processed them for you.
2) Never mention internal markers or identifiers from the retrieved context (such as IDs, scores, or file paths). They are for internal use only.
3) Use the retrieved context when (and only when) it is relevant to the user's question.
4) If the answer is not present in the retrieved context nor in the conversation history, say so explicitly instead of inventing details.
5) Focus on the user's intent rat

## 5) One-shot ingestion for this session

The vector store already contains data from previous experiments, but the
`ContextBuilder` uses **session-scoped retrieval** (filters by `session_id`,
`user_id`, etc.).

To make RAG work for the **current** session, we need to:

1. Recreate the same `AttachmentRef` as in the ingestion notebook
   (e.g. for `PROJECT_STRUCTURE.md`).
2. Send a `RuntimeRequest` with this attachment to `DropInKnowledgeRuntime.ask(...)`.
   - The runtime will:
     - store a user message with the attachment in the session,
     - run the ingestion pipeline,
     - push chunks to the vector store with this session’s metadata.

We do this as a **single, compact step** – the goal of this notebook
is still to test `ContextBuilder`, not to re-explain the ingestion pipeline.


In [ ]:
from intergrax.llm.messages import AttachmentRef
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest

# 5.1 Recreate the attachment used in the ingestion demo
project_root_file = Path("../../PROJECT_STRUCTURE.md").resolve()

if not project_root_file.exists():
    raise FileNotFoundError(f"Expected demo file does not exist: {project_root_file}")

attachment_uri = project_root_file.as_uri()

attachment = AttachmentRef(
    id="project-structure-md",
    type="markdown",
    uri=attachment_uri,
    metadata={
        "description": "Intergrax project structure document",
        "scope": "intergrax-docs-demo",
    },
)

print("AttachmentRef recreated:")
print("  id       :", attachment.id)
print("  type     :", attachment.type)
print("  uri      :", attachment.uri)


# 5.2 Send a one-shot request with this attachment to the runtime
# NOTE:
# This call should trigger ingestion for THIS session:
# - the engine stores the message + attachment in the session store,
# - the ingestion service resolves the file, chunks it, embeds it,
#   and upserts vectors into the configured vector store.

ingestion_request = RuntimeRequest(
    session_id=session.id,
    user_id=session.user_id,
    tenant_id=session.tenant_id,
    workspace_id=session.workspace_id,
    message="Please index this attachment for the current session.",
    attachments=[attachment],
)

print("\nSending ingestion request to DropInKnowledgeRuntime...")
ingestion_answer = await runtime.run(ingestion_request)

print("Ingestion request completed.")
print("Answer summary:", ingestion_answer.status)

# 5.3 Reload session to ensure the new message (with attachment) is stored
session = await session_manager.get_session(session.id)
messages = await session_manager.get_history(session_id=session.id)
print("\nSession now has", len(messages), "message(s).")


AttachmentRef recreated:
  id       : project-structure-md
  type     : markdown
  uri      : file:///D:/Projekty/intergrax/PROJECT_STRUCTURE.md

Sending ingestion request to DropInKnowledgeRuntime...
[intergraxVectorstoreManager] Upserting 100 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 100
Ingestion request completed.
Answer summary: ok

Session now has 6 message(s).
